Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/sysid.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='560c2adace05eb20ebd78377582015d5b2d3859a', drake_version='0.27.0', drake_build='release')

import numpy as np
import matplotlib.pyplot as plt

import pydrake.all
from pydrake.all import (
    LinearSystem,
    DiagramBuilder, PiecewisePolynomial, TrajectorySource,
    LogOutput, Simulator, LinearQuadraticRegulator
)
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve


# Linear System Identification

In this notebook, you will test different objective functions in linear system identification. Consider a discrete-time linear system of the form

$$x[n+1] = Ax[n]+Bu[n]$$
where $x[n]\in\mathbf{R}^p$ and $u[n]\in\mathbf{R}^q$ are state and control at step $n$.
The system matrix $A\in\mathbf{R}^{p\times p}$ and $B\in\mathbf{R}^{p\times q}$ are unknown parameters of the model, and your task is to identify the parameters given a simulated trajectory.

Let's first define a test system:

In [ ]:
A = np.array([[0.8, -0.1, 0.0, 0.0], 
              [0., 0.2, 0.0, 0.1], 
              [0., -0.1, 1.0001, 0.0], 
              [0., 0.2, 0.0, 0.1]])

B = np.array([[0.0], [-0.02], [0.01], [-0.02]])
C = np.identity(4)
D = np.zeros((4,1))

Let's run a simulation to obtain the trajectory data. We add noise to the state representing measurement noise.

In [ ]:
def generate_data(A, B, C, D, noise_level = 1e-5):
    # Define a Linear system
    sys = LinearSystem(A, B, C, D, time_period = 0.1)

    ts = np.arange(0, 100, 0.1)
    utraj = PiecewisePolynomial.CubicShapePreserving(ts, np.sin(0.1*ts).reshape((1,-1)))

    builder = DiagramBuilder()
    plant = builder.AddSystem(sys)

    usys = builder.AddSystem(TrajectorySource(utraj)) 
    builder.Connect(usys.get_output_port(), plant.get_input_port())

    logger = LogOutput(plant.get_output_port(), builder)
    diagram = builder.Build()

    simulator = Simulator(diagram)
    simulator.AdvanceTo(ts[-1])

    t = logger.sample_times()
    U = utraj.vector_values(logger.sample_times())
    X = logger.data()
    X = X + np.random.normal(size = X.shape)*noise_level

    return t, U, X
        

t, U, X = generate_data(A, B, C, D)


fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(t, X.T)
ax[1].plot(t, U.T)
ax[0].set_title("state")
ax[1].set_title("control")
ax[0].set_xlabel("time")
ax[1].set_xlabel("time")
plt.show()

The output of the simulation gives the data matrices in the following format:

$$X=\begin{bmatrix} \lvert && \lvert && && \lvert \\ x[0] && x[1] && \dots && x[N] \\ \lvert && \lvert && && \lvert \end{bmatrix}$$


$$U=\begin{bmatrix} \lvert && \lvert && && \lvert \\ u[0] && u[1] && \dots && u[N] \\ \lvert && \lvert && && \lvert \end{bmatrix}.$$

Using the simulated data, you will implement regression algorithms to identify the system model.

(a) Identify the model by solving the following optimization problem: 

$$\min_{A, B}\sum_{n=0}^{N-1}\lVert x[n+1] - Ax[n] -Bu[n] \rVert_2^2$$

The function should output the matrices $A$ and $B$ that are the solutions to the above optimization problem.

In [ ]:
def sysid_2norm(X, U):
  n = X.shape[0]
  m = U.shape[0]
  A = np.zeros((n,n))
  B = np.zeros((n,m))
  ######### Put your Solution here ############


  ##############################################
  return A, B

Ahat_2norm, Bhat_2norm = sysid_2norm(X, U)

print("A = ")
print(A)
print("Â = ")
print(Ahat_2norm)
print("")
print("B = ")
print(B)
print("B̂ = ")
print(Bhat_2norm)
print("")

res = Ahat_2norm @ X[:,:-1] + Bhat_2norm @ U[:,:-1] - X[:,1:]
print(f'residual (2-norm): {np.sum(res**2)}')
print(f'residual (inf-norm): {np.sum(np.max(np.abs(res), axis = 0))}')
print(f'residual (1-norm): {np.sum(np.abs(res))}')

(b) Identify the model by solving the following optimization problem: 

$$\min_{A, B}\sum_{n=0}^{N-1}\lVert x[n+1] - Ax[n] -Bu[n] \rVert_{\infty}.$$

where $L\infty$ norm of a vector is defined as

$$\lVert x \rVert_\infty = \max_{1\leq i\leq p}\lvert x_i\rvert.$$
Implement the following function that outputs the matrices $A$ and $B$ that are the solutions to the above optimization problem.


In [ ]:
def sysid_infnorm(X, U):
  n = X.shape[0]
  m = U.shape[0]
  A = np.zeros((n,n))
  B = np.zeros((n,m))
  ######### Put your Solution here ############

  prog = MathematicalProgram()
  result = Solve(prog)
  obj_value = result.get_optimal_cost()

  ##############################################
  return A, B, obj_value

Ahat_infnorm, Bhat_infnorm, obj_infnorm = sysid_infnorm(X, U)

print("A = ")
print(A)
print("Â = ")
print(Ahat_infnorm)
print("")
print("B = ")
print(B)
print("B̂ = ")
print(Bhat_infnorm)
print("")

res = Ahat_infnorm @ X[:,:-1] + Bhat_infnorm @ U[:,:-1] - X[:,1:]
print(f'residual (2-norm): {np.sum(res**2)}')
print(f'residual (inf-norm): {np.sum(np.max(np.abs(res), axis = 0))}')
print(f'residual (1-norm): {np.sum(np.abs(res))}')

(c) Identify the model by solving the following optimization problem: 

$$\min_{A, B}\sum_{n=0}^{N-1}\lVert x[n+1] - Ax[n] -Bu[n] \rVert_1.$$
where the $L1$ norm of a vector is defined as
$$\lVert x \rVert_1 = \sum_{i=1}^p \lvert x_i\rvert.$$
The function should output the matrices $A$ and $B$ that are the solutions to the above optimization problem.


In [ ]:
def sysid_1norm(X, U):
  n = X.shape[0]
  m = U.shape[0]
  A = np.zeros((n,n))
  B = np.zeros((n,m))
  ######### Put your Solution here ############

  prog = MathematicalProgram()
  result = Solve(prog)
  obj_value = result.get_optimal_cost()
    
  ##############################################
  return A, B, obj_value

Ahat_1norm, Bhat_1norm, obj_1norm = sysid_1norm(X, U)

print("A = ")
print(A)
print("Â = ")
print(Ahat_1norm)
print("")
print("B = ")
print(B)
print("B̂ = ")
print(Bhat_1norm)
print("")

res = Ahat_1norm @ X[:,:-1] + Bhat_1norm @ U[:,:-1] - X[:,1:]
print(f'residual (2-norm): {np.sum(res**2)}')
print(f'residual (inf-norm): {np.sum(np.max(np.abs(res), axis = 0))}')
print(f'residual (1-norm): {np.sum(np.abs(res))}')

Let's try LQR controller with the identified models. Using the estimated parameters, $\hat{A}$ and $\hat{B}$, we solve discrete-time LQR. The controller is used to stabilize the original linear system defined by the parameters, $A$ and $B$. If the estimate was accurate, we expect the controller to work well for the original system.

In [ ]:

def test_LQR(A_hat, B_hat, Q = None, R = None):
  Q = np.identity(A.shape[0]) if Q is None else Q
  R = np.identity(B.shape[1]) if R is None else R
  
  # Define a Linear system
  sys = LinearSystem(A, B, C, D, time_period = 0.1)
  sys_hat = LinearSystem(A_hat, B_hat, C, D, time_period = 0.1)

  builder = DiagramBuilder()
  plant = builder.AddSystem(sys)

  controller = builder.AddSystem(LinearQuadraticRegulator(sys_hat, Q, R))

  builder.Connect(controller.get_output_port(), plant.get_input_port())
  builder.Connect(plant.get_output_port(), controller.get_input_port())

  logger = LogOutput(plant.get_output_port(), builder)
  log_ctrl = LogOutput(controller.get_output_port(), builder)

  diagram = builder.Build()
  simulator = Simulator(diagram)
  context = simulator.get_mutable_context()
  context.SetDiscreteState(0,[10.,5.,5.,1.])

  simulator.AdvanceTo(50)

  t = logger.sample_times()
  U = log_ctrl.data()
  X = logger.data()

  return t, U, X

t, U_2norm, X_2norm = test_LQR(Ahat_2norm, Bhat_2norm)
t, U_infnorm, X_infnorm = test_LQR(Ahat_infnorm, Bhat_infnorm)
t, U_1norm, X_1norm = test_LQR(Ahat_1norm, Bhat_1norm)

fig, ax = plt.subplots(2, 3, figsize=(16, 8))
ax[0,0].plot(t, X_2norm.T)
ax[1,0].plot(t, U_2norm.T)
ax[0,1].plot(t, X_infnorm.T)
ax[1,1].plot(t, U_infnorm.T)
ax[0,2].plot(t, X_1norm.T)
ax[1,2].plot(t, U_1norm.T)
ax[0,0].set_ylabel("state")
ax[1,0].set_ylabel("control")
ax[0,0].set_title("2-norm solution")
ax[0,1].set_title("$\infty$-norm solution")
ax[0,2].set_title("1-norm solution")
plt.show()


You can also test the model with increased noise in the data. You can play around with different values of 'noise_level' to vary the intensity of the noise in the measurement.

In [ ]:
t, U_noisy, X_noisy = generate_data(A, B, C, D, noise_level = 0.5)

Ahat_noisy_2norm, Bhat_noisy_2norm = sysid_2norm(X_noisy, U_noisy)
t, U_2norm, X_2norm = test_LQR(Ahat_noisy_2norm, Bhat_noisy_2norm)

Ahat_noisy_infnorm, Bhat_noisy_infnorm, obj_infnorm_noisy = sysid_infnorm(X_noisy, U_noisy)
t, U_infnorm, X_infnorm = test_LQR(Ahat_noisy_infnorm, Bhat_noisy_infnorm)

Ahat_noisy_1norm, Bhat_noisy_1norm, obj_1norm_noisy = sysid_1norm(X_noisy, U_noisy)
t, U_1norm, X_1norm = test_LQR(Ahat_noisy_1norm, Bhat_noisy_1norm)

fig, ax = plt.subplots(2, 3, figsize=(16, 8))
ax[0,0].plot(t, X_2norm.T)
ax[1,0].plot(t, U_2norm.T)
ax[0,1].plot(t, X_infnorm.T)
ax[1,1].plot(t, U_infnorm.T)
ax[0,2].plot(t, X_1norm.T)
ax[1,2].plot(t, U_1norm.T)
ax[0,0].set_ylabel("state")
ax[1,0].set_ylabel("control")
ax[0,0].set_title("2-norm solution")
ax[0,1].set_title("$\infty$-norm solution")
ax[0,2].set_title("1-norm solution")
plt.show()


## Autograding
You can check your work by running the following cell:

In [ ]:
from underactuated.exercises.sysid.linear_sysid.test_linear_sysid import TestLinearSysid
from underactuated.exercises.grader import Grader
Grader.grade_output([TestLinearSysid], [locals()], 'results.json')
Grader.print_test_results('results.json')
